In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torchvision.transforms import v2

# GPU 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 하이퍼파라미터
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 0.001
IMG_SIZE = (224, 224)

# 데이터셋 경로 설정
DATA_DIR = r'C:\Users\User\AppData\Local\Programs\Python\KDT-2024-main\TORCH_DL\PROJECT\OVERWATCH'

# 데이터 변환 설정 (가우시안 블러 추가)
transConvert = v2.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.RandomRotation(10),  # 10도 회전
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # 색상 변환
    transforms.RandomApply([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomAdjustSharpness(0)
    ], p=0.8),
    transforms.RandomApply([
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))  # 랜덤 가우시안 블러 추가
    ], p=0.5),  # 50% 확률로 가우시안 블러 적용
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ImageFolder로 이미지 로드
imgDS = ImageFolder(root=DATA_DIR, transform=transConvert)

# 데이터셋을 80% 훈련, 20% 검증으로 분할
train_size = int(0.8 * len(imgDS))
val_size = len(imgDS) - train_size

train_dataset, val_dataset = random_split(imgDS, [train_size, val_size])

# DataLoader로 데이터셋 로드
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# ResNet50 모델 불러오기 (ImageNet으로 사전 학습된 모델 사용)
model = models.resnet50(pretrained=True)

# 출력 레이어 수정 (클래스 수에 맞게)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(imgDS.classes))  # 클래스 개수에 맞게 수정

# 모델을 GPU로 이동
model = model.to(device)

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 학습 함수
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    total_correct = 0
    total_samples = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # 손실과 정확도 계산
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    epoch_loss = running_loss / total_samples
    epoch_acc = total_correct / total_samples

    return epoch_loss, epoch_acc

# 검증 함수
def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)

    epoch_loss = running_loss / total_samples
    epoch_acc = total_correct / total_samples

    return epoch_loss, epoch_acc

# 학습 실행
for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)

    print(f'Epoch {epoch+1}/{EPOCHS}')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}')
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}')

# 학습된 모델 저장
torch.save(model.state_dict(), 'resnet50_custom_dataset_with_gaussian_blur.pth')


Epoch 1/10
Train Loss: 0.1368, Train Accuracy: 0.9542
Validation Loss: 0.0503, Validation Accuracy: 0.9844
Epoch 2/10
Train Loss: 0.0563, Train Accuracy: 0.9821
Validation Loss: 0.0353, Validation Accuracy: 0.9880
Epoch 3/10
Train Loss: 0.0316, Train Accuracy: 0.9900
Validation Loss: 0.0196, Validation Accuracy: 0.9936
Epoch 4/10
Train Loss: 0.0252, Train Accuracy: 0.9921
Validation Loss: 0.0626, Validation Accuracy: 0.9813
Epoch 5/10
Train Loss: 0.0210, Train Accuracy: 0.9932
Validation Loss: 0.0024, Validation Accuracy: 0.9993
Epoch 6/10
Train Loss: 0.0177, Train Accuracy: 0.9945
Validation Loss: 0.0156, Validation Accuracy: 0.9951
Epoch 7/10
Train Loss: 0.0171, Train Accuracy: 0.9949
Validation Loss: 0.0076, Validation Accuracy: 0.9976
Epoch 8/10
Train Loss: 0.0155, Train Accuracy: 0.9954
Validation Loss: 0.0093, Validation Accuracy: 0.9970
Epoch 9/10
Train Loss: 0.0115, Train Accuracy: 0.9966
Validation Loss: 0.0149, Validation Accuracy: 0.9952
Epoch 10/10
Train Loss: 0.0113, Train

In [3]:
# 학습된 모델 저장
torch.save(model.state_dict(), 'resnet50_OVERWATCH.pth')
